# Load the necessary libraries

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Load the data and preprocess it

In [ ]:
# === 1️⃣ LOAD AND PREPARE DATA ===
merged_data_df_cv = pd.read_csv("combined_monthly_data_cv_good.csv", parse_dates=['time'])
data_clean = merged_data_df_cv.dropna(subset=['CHL'])
data_clean['month'] = data_clean['time'].dt.month

# Group by month (CHL, SST, Wind, AOD)
monthly_stats = data_clean.groupby('month')[['CHL', 'TS_C', 'wind_speed', 'AODANA']].agg(['mean', 'std']).reset_index()

# Load deposition dataset
cv_dry_wet_df = pd.read_csv("CV_DEPOSITIONS/total_deposition.csv", parse_dates=['time'])
cv_dry_wet_df['month'] = cv_dry_wet_df['time'].dt.month
cv_dry_wet_monthly = cv_dry_wet_df.groupby('month')[['total_wet_deposition', 'total_dry_deposition']].agg(['mean', 'std']).reset_index()

# === 2️⃣ PLOT SETTINGS ===
sns.set_theme(style="whitegrid", context="paper")
plt.rcParams.update({
    'font.family': 'Times New Roman',
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11,
    'legend.fontsize': 10,
    'grid.linestyle': '--',
    'grid.alpha': 0.6
})

month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Define line styles
styles = {
    'chl': dict(color='#1f77b4', lw=2.2, marker='o', markersize=6, label='CHL (mg/m³)'),
    'temp': dict(color='#d62728', lw=2.2, marker='s', markersize=6, label='SST (°C)'),
    'wind': dict(color='#2ca02c', lw=2.2, marker='^', markersize=6, label='Wind Speed (m/s)'),
    'aod': dict(color='#9467bd', lw=2.2, marker='D', markersize=6, label='AOD (550 nm)'),
    'wet': dict(color='#0e29b3', lw=2.2, marker='x', markersize=6, label='Wet Deposition'),
    'dry': dict(color='#ff0e0e', lw=2.2, marker='P', markersize=6, label='Dry Deposition')
}

# === 3️⃣ SEASONAL COLOR SHADING FUNCTION ===
# def add_season_shading(ax):
#     seasons = {
#         'DJF': ([12, 1, 2], "#e0fff2"),
#         'MAM': ([3, 4, 5], '#eaf7e0'),
#         'JJA': ([6, 7, 8], '#fff3cd'),
#         'SON': ([9, 10, 11], '#ffe0e0')
#     }
#     for _, (months, color) in seasons.items():
#         for m in months:
#             ax.axvspan(m - 0.5, m + 0.5, color=color, alpha=0.3, lw=0)

# === 4️⃣ CREATE 2×2 SUBPLOT ===
fig, axes = plt.subplots(2, 2, figsize=(15, 9), dpi=360, sharex=True)
plt.subplots_adjust(hspace=0.3, wspace=0.25)

# Flatten axes for easy indexing
axes = axes.flatten()

# === 1️⃣ CHL vs SST ===
ax1 = axes[0]
# add_season_shading(ax1)
ax1.fill_between(monthly_stats['month'],
                 monthly_stats[('CHL', 'mean')] - monthly_stats[('CHL', 'std')],
                 monthly_stats[('CHL', 'mean')] + monthly_stats[('CHL', 'std')],
                 color=styles['chl']['color'], alpha=0.2)
chl_line, = ax1.plot(monthly_stats['month'], monthly_stats[('CHL', 'mean')], **styles['chl'])
ax1.set_ylabel('CHL (mg/m³)')
ax1_twin = ax1.twinx()
ax1_twin.fill_between(monthly_stats['month'],
                      monthly_stats[('TS_C', 'mean')] - monthly_stats[('TS_C', 'std')],
                      monthly_stats[('TS_C', 'mean')] + monthly_stats[('TS_C', 'std')],
                      color=styles['temp']['color'], alpha=0.2)
temp_line, = ax1_twin.plot(monthly_stats['month'], monthly_stats[('TS_C', 'mean')], **styles['temp'])
ax1_twin.set_ylabel('SST (°C)')
ax1.set_title('CHL vs SST')
ax1.legend([chl_line, temp_line], [chl_line.get_label(), temp_line.get_label()],
           loc='upper right', frameon=False)

# === 2️⃣ CHL vs Wind ===
ax2 = axes[1]
# add_season_shading(ax2)
ax2.fill_between(monthly_stats['month'],
                 monthly_stats[('CHL', 'mean')] - monthly_stats[('CHL', 'std')],
                 monthly_stats[('CHL', 'mean')] + monthly_stats[('CHL', 'std')],
                 color=styles['chl']['color'], alpha=0.2)
ax2.plot(monthly_stats['month'], monthly_stats[('CHL', 'mean')], **styles['chl'])
ax2.set_ylabel('CHL (mg/m³)')
ax2_twin = ax2.twinx()
ax2_twin.fill_between(monthly_stats['month'],
                      monthly_stats[('wind_speed', 'mean')] - monthly_stats[('wind_speed', 'std')],
                      monthly_stats[('wind_speed', 'mean')] + monthly_stats[('wind_speed', 'std')],
                      color=styles['wind']['color'], alpha=0.2)
wind_line, = ax2_twin.plot(monthly_stats['month'], monthly_stats[('wind_speed', 'mean')], **styles['wind'])
ax2_twin.set_ylabel('Wind Speed (m/s)')
ax2.set_title('CHL vs Wind Speed')
ax2.legend([chl_line, wind_line], [chl_line.get_label(), wind_line.get_label()],
           loc='upper right', frameon=False)

# === 3️⃣ CHL vs AOD ===
ax3 = axes[2]
# add_season_shading(ax3)
ax3.fill_between(monthly_stats['month'],
                 monthly_stats[('CHL', 'mean')] - monthly_stats[('CHL', 'std')],
                 monthly_stats[('CHL', 'mean')] + monthly_stats[('CHL', 'std')],
                 color=styles['chl']['color'], alpha=0.2)
ax3.plot(monthly_stats['month'], monthly_stats[('CHL', 'mean')], **styles['chl'])
ax3.set_ylabel('CHL (mg/m³)')
ax3_twin = ax3.twinx()
ax3_twin.fill_between(monthly_stats['month'],
                      monthly_stats[('AODANA', 'mean')] - monthly_stats[('AODANA', 'std')],
                      monthly_stats[('AODANA', 'mean')] + monthly_stats[('AODANA', 'std')],
                      color=styles['aod']['color'], alpha=0.2)
aod_line, = ax3_twin.plot(monthly_stats['month'], monthly_stats[('AODANA', 'mean')], **styles['aod'])
ax3_twin.set_ylabel('AOD (550 nm)')
ax3.set_title('CHL vs AOD')
ax3.legend([chl_line, aod_line], [chl_line.get_label(), aod_line.get_label()],
           loc='upper right', frameon=False)

# === 4️⃣ Dry vs Wet Deposition ===
ax4 = axes[3]
# add_season_shading(ax4)
ax4.fill_between(cv_dry_wet_monthly['month'],
                 cv_dry_wet_monthly[('total_wet_deposition', 'mean')] - cv_dry_wet_monthly[('total_wet_deposition', 'std')],
                 cv_dry_wet_monthly[('total_wet_deposition', 'mean')] + cv_dry_wet_monthly[('total_wet_deposition', 'std')],
                 color=styles['wet']['color'], alpha=0.2)
wet_line, = ax4.plot(cv_dry_wet_monthly['month'], cv_dry_wet_monthly[('total_wet_deposition', 'mean')],
                     **styles['wet'])
ax4.set_ylabel('Wet Deposition (kg/m²/s)')
ax4_twin = ax4.twinx()
ax4_twin.fill_between(cv_dry_wet_monthly['month'],
                      cv_dry_wet_monthly[('total_dry_deposition', 'mean')] - cv_dry_wet_monthly[('total_dry_deposition', 'std')],
                      cv_dry_wet_monthly[('total_dry_deposition', 'mean')] + cv_dry_wet_monthly[('total_dry_deposition', 'std')],
                      color=styles['dry']['color'], alpha=0.2)
dry_line, = ax4_twin.plot(cv_dry_wet_monthly['month'], cv_dry_wet_monthly[('total_dry_deposition', 'mean')],
                          **styles['dry'])
ax4_twin.set_ylabel('Dry Deposition (kg/m²/s)')
ax4.set_title('d)')
ax4.legend([wet_line, dry_line], [wet_line.get_label(), dry_line.get_label()],
           loc='upper right', frameon=False)

# === COMMON FORMATTING ===
for ax in axes:
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(month_labels)
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

plt.tight_layout(pad=2)
# plt.savefig('seasonal_variations_2x2_with_shading.png', dpi=360, bbox_inches='tight', facecolor='white')
plt.show()
